In [1]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse


#Проверяем, является ли url действительным URL
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


#Возвращает все URL‑адреса изображений по одному `url`
def get_all_images(url):
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Получено изображение"):
        img_url = img.attrs.get("src")
        if not img_url:
            # если img не содержит атрибута src, просто пропускаем
            continue
        # сделаем URL абсолютным, присоединив имя домена к только что извлеченному URL
        img_url = urljoin(url, img_url)
        # удалим URL‑адреса типа '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # наконец, если URL действителен
        if is_valid(img_url):
            urls.append(img_url)
    return urls


#Загружает файл по URL‑адресу и помещает его в папку `pathname`
def download(url, pathname):
    # если путь не существует, создать dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # загружаем тело ответа по частям, а не сразу
    response = requests.get(url, stream=True)

    # получить общий размер файла
    file_size = int(response.headers.get("Content-Length", 0))

    # получаем имя файла
    filename = os.path.join(pathname, url.split("/")[-1])

    # индикатор выполнения, изменение единицы измерения на байты вместо итераций (по умолчанию tqdm)
    progress = tqdm(response.iter_content(1024), f"Загружен {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # записываем прочитанные данные в файл
            f.write(data)
            # обновление индикатора выполнения вручную
            progress.update(len(data))


def main(url, path):
    # получить все изображения
    imgs = get_all_images(url)
    for img in imgs:
        # скачать для каждого img
        download(img, path)
    



In [2]:
main("https://unsplash.com/t/animals", "C:/Users/Пользователь/Desktop/Изображения")

Получено изображение: 100%|██████████| 164/164 [00:00<00:00, 41261.22it/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\p: 100%|██████████| 43.0/43.0 [00:00<00:00, 8.60kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1696226100906-d9289cd6eb64: 100%|██████████| 571k/571k [00:19<00:00, 30.4kB/s] 
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1583427783052-3da8ceab5579image: 100%|██████████| 814k/814k [00:35<00:00, 23.6kB/s] 
Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1695765586912-39758d5de97d: 100%|██████████| 5.34M/5.34M [06:12<00:00, 15.0kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1583427783052-3da8ceab5579image: 100%|██████████| 814k/814k [00:26<00:00, 31.8kB/s] 
Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1696024344604-46b33ba2c753: 100%|██████████| 1.79M/1.79M [02:46<00:00, 11.2kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1516997253075-2a25da8007e7: 100%|██████████| 150k/150k [00:0

KeyboardInterrupt: 

In [3]:
import zipfile, os, re
 
path = r'C:/Users/Пользователь/Desktop/Изображения/'
pattern = r''
path_to_archive = r'test.zip'
all_files = []
 
with zipfile.ZipFile(path_to_archive, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for root_path, dirs, files in os.walk(path):
        for name in files:
            full_path = os.path.join(root_path, name)
            if os.path.isfile(full_path) and re.match(pattern, name):
                if root_path == path:
                    zf.write(full_path, name)
                else:
                    res = os.path.split(root_path)[-1]
                    zf.write(full_path, os.path.join(res, name))
                all_files.append(full_path)
 
print(os.path.abspath(path_to_archive))
print('\n'.join(all_files))

Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1684867070626-1fad1f165d24:  93%|█████████▎| 2.54M/2.75M [01:59<00:16, 12.7kB/s]

c:\Users\Пользователь\Desktop\Анализ веб данных\test.zip
C:/Users/Пользователь/Desktop/Изображения/extra-large.jpg
C:/Users/Пользователь/Desktop/Изображения/p
C:/Users/Пользователь/Desktop/Изображения/photo-1684867070626-1fad1f165d24
C:/Users/Пользователь/Desktop/Изображения/photo-1686367134749-6795adf7f2ec
C:/Users/Пользователь/Desktop/Изображения/photo-1687692414808-be94eadc8d44
C:/Users/Пользователь/Desktop/Изображения/photo-1689232535234-79db492ed871
C:/Users/Пользователь/Desktop/Изображения/photo-1695765586912-39758d5de97d
C:/Users/Пользователь/Desktop/Изображения/photo-1696024344604-46b33ba2c753
C:/Users/Пользователь/Desktop/Изображения/photo-1696199246101-73011737f8ce
C:/Users/Пользователь/Desktop/Изображения/photo-1696199386921-0f8f7b03973d
C:/Users/Пользователь/Desktop/Изображения/photo-1696203084713-d294963add53
C:/Users/Пользователь/Desktop/Изображения/photo-1696226100906-d9289cd6eb64
C:/Users/Пользователь/Desktop/Изображения/photo-1696230390138-75932d745228
C:/Users/Пользов